# **Desafio Tecnico: Mercado Libre**

**Consignas**:
- 1) Realizar un analisis exploratorio de las publicaciones con descuento del marketplace
- 2) Armar un dataset y un modelo que permita predecir con atributos de la publicacion el valor de `sold quantity`

**Aclaracion:**
En un primer momento opte armar un modelo general que permita predecir cualquier tipo de publicacion. El problema con este enfoque era que involucraba una complejidad de infraestructura y extension de tiempo que, dada la escasez, enfrente pero decidi abandonar. 

Es por esta razon que decidi limitar el enfoque a predecir 5 tipo de productos cuyas caracteristicas estaban bien definidas y tenian muchas publicaciones: `computadoras`, `monitores`, `parlantes`, `notebooks` y `celulares`. Television fue una opcion pero el endpoint de search devolvia mas resultados de TV_STANDS que de TV.

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from lightgbm import LGBMRegressor

from utils import *

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
token = get_token(APP_INFO["app_id"], APP_INFO["secret_key"], AUTH["refresh_token"])

In [ ]:
productos = ["notebook", "celular", "monitor", "parlante", "auriculares"]
final_df = get_dataset(productos, token)

## **EDA**

In [ ]:
# Creamos las columnas con los descuentos
final_df['has_discount'] = final_df['original_price'].notnull()
final_df['discount'] = ((final_df['original_price'] - final_df['price'])/ final_df['original_price']).fillna(0)

Observemos la distribucion general de descuentos

In [ ]:
sns.distplot(final_df.query('discount != 0')['discount'])

Podemos ver que el grueso esta entre 15% y 35%

Ahora, veamos la proporcion de productos que tienen descuentos agrupado por producto

In [ ]:
props = final_df.groupby(["domain_id"])['has_discount'].value_counts(normalize=True)
props_df = props.to_frame()
props_df = props_df.rename(columns={'has_discount': 'prop'}).reset_index().sort_values('prop', ascending=False)
filter_props = props_df.query("has_discount == True")
sns.barplot(x="domain_id", y="prop", color="salmon", data=filter_props).set_title("Proporcion de publicaciones con descuento")
plt.xticks(rotation=60);

No pareceria haber muchas publicaciones con descuento, lo maximo es 5% para parlantes y ~4% para auriculares

Veamos de los productos con descuentos, cual es el descuento promedio

In [ ]:
discounts = final_df.query('discount != 0')
discounts = discounts.groupby('domain_id').mean().reset_index().sort_values('discount', ascending=False)
g = sns.barplot(x="domain_id", y="discount", data=discounts, color="salmon").set_title("Descuento promedio")
plt.xticks(rotation=60);

De todas maneras, el descuento promedio es mayor al 20% para todos los productos

Dado que el grafico anterior era una estimacion puntual, tratemos de ver la distribucion de descuentos para cada producto

En histograma

In [ ]:
final_df.query('discount != 0')['discount'].hist(by=final_df['domain_id'], figsize=(15,15));

En densidad:

In [ ]:
plt.figure(figsize=(15,10))
sns.set_style("darkgrid")
for main_category in final_df['domain_id'].unique():
    temp = final_df.query('discount != 0')
    temp = temp.loc[temp['domain_id'] == main_category]
    sns.distplot(temp[['discount']], hist=False, label=main_category)


Es llamativamente interesante como la mayoria de los descuentos para las notebboks se centra en 30% mientras que el resto se distribuye mas uniformemente entren el 10% y el 35%

Veamoslo en un boxplot:

In [ ]:
final_df.query('discount != 0').boxplot(column=['discount'], by='domain_id', figsize=(15,10), rot=60, color=dict(medians='r'));

# **Data Wrangling**

Antes de procesar las columnas con formatos no convencionales, tratemos de limpiar un poco nuestro dataset. Se busca eliminar:
- Las features que tienen valores unicos para cada observacion
- Feature que tienen el mismo valor para todas las observaciones
- Observaciones con outliers en nuestras variables numericas

In [ ]:
final_df.columns

Por simplicidad, eliminemos las variales espaciales latitud y longitud

In [ ]:
final_df.drop(["seller_address.latitude", "seller_address.longitude", "geolocation.latitude", "geolocation.longitude"], axis=1, inplace=True)

Podemos ver que las variables price y base_price parecen ser iguales, veamoslo...

In [ ]:
sum(final_df['price'] != final_df['base_price'])

In [ ]:
final_df.drop("base_price", axis=1, inplace=True)

Entonces, nuestras variables numericas son:
- price
- original_price
- initial_quantity
- available_quantity
- sold_quantity
- discount

In [ ]:
numeric_features = ["price", "discount", "available_quantity", "initial_quantity", "original_price", "sold_quantity"]
final_df[numeric_features].hist(figsize=(12,8));

In [ ]:
final_df[numeric_features].describe()

Bueno, tenemos valorex extremos en casi todas las categorias... arranquemos por analizar los precios

In [ ]:
final_df.query("price > 1e05")[['title', 'price']].sort_values('price', ascending=False)[:20]

Podemos ver que estos precios son llamativos, sobretodo el precio de "La camperita termica para perro". Podriamos hacer un analisis para determinar cual deberia ser el precio que deberia corresponder a estos items, pero voy a optar por eliminarlos. Son las primeras 12 observaciones (hasta "celular") que parecen incorrectas, el resto parecerian estar relacionados al producto que venden

In [ ]:
final_df = final_df[final_df['price'] < 9954695]

Ahora observemos las cantidades vendidas

In [ ]:
final_df[['title', 'sold_quantity', "date_created"]].sort_values('sold_quantity', ascending=False)[:20]

Parecerian ser todos numeros plausibles dada la antiguedad de las publicaciones, sigamos.

Ahora observemos las cantidades en stock

In [ ]:
final_df[["title", "seller_id", "available_quantity"]].sort_values("available_quantity", ascending=False)[:100]

Pareceria ser que son casi todas las publicaciones de valores extremos pertenencen al mismo vendedor. Me llama la atencion que dispongan de semejante volumen aunque no parecen valores descabellados, creo. Ante la duda voy a optar por dejarlos. 

Ahora analizemos la variabilidad de nuestras features categoricas

In [ ]:
categorical_features = list(final_df.drop(numeric_features, axis=1).columns)

In [ ]:
uniques ={}
for col in categorical_features:
    uniques[col] = final_df[col].apply(str).nunique() / len(final_df)

Empecemos viendo las que serian unicas por publicacion

In [ ]:
sorted(uniques.items(), key=lambda x: x[1], reverse=True)[:15]

Aca aparecen muchas de las variables unicas (id, permalink, descriptions, pictures, thumbnail, secure_thumbnail) que considero hay que eliminar, pero tambien una de las variables que en mi opinion, mas importa para predecir la cantidad de articulos vendidos, date_created. Veamos si start_time, historical_start_time y date_created son iguales

In [ ]:
final_df.drop(["id", "permalink", "thumbnail", "secure_thumbnail", "descriptions", "pictures"], axis=1, inplace=True)

In [ ]:
final_df[final_df['start_time'] != final_df['date_created']][['start_time', "date_created", "historical_start_time"]]

Los casos en que no son iguales, que son pocos, tienen 1 segundo de diferencia, elimiemos start_time y historical_start_time

In [ ]:
final_df.drop(['start_time', "historical_start_time"], axis=1, inplace=True)

Y ahora transformemos un poco las columnas que tienen fechas

In [ ]:
final_df['days_old'] = (pd.Timestamp(datetime.utcnow(), tz="UTC") - pd.to_datetime(final_df['date_created'])).dt.days
final_df['days_remaining'] = (pd.to_datetime(final_df['stop_time']) - pd.Timestamp(datetime.utcnow(), tz="UTC")).dt.days
final_df['days_from_update'] = (pd.Timestamp(datetime.utcnow(), tz="UTC") - pd.to_datetime(final_df['last_updated'])).dt.days

final_df.drop(["date_created", "stop_time", "last_updated"], axis=1, inplace=True)

Ahora observemos si hay variables que tienen valores unicos

In [ ]:
sorted(uniques.items(), key=lambda x: x[1])[:20]

Las variables cuyo valor es 2.79..e-05 tienen un solo valor. Eliminsemoslas dado que no aportan ningun tipo de informacion

In [ ]:
keys = [key for key, value in sorted(uniques.items(), key=lambda x: x[1])[:17]]

In [ ]:
final_df.drop(keys, axis=1, inplace = True)

Ahora, transformemos y adaptemos un poco las variables que tienen texto adentro como por ejemplo el titulo del la publicacion y la ciudad del vendedor

Para el titulo, quedemosnos pasemos todo a minusculas, eliminemos whitespaces y quedemosnos solo con las primeras dos palabras que usualmente tienen el nombre del producto y su marca

In [ ]:
def transform_title(x):
    words = x.strip().lower().split(" ")[:2]
    parsed_words = " ".join(words)
    return parsed_words

final_df.loc[:, 'title'] = final_df["title"].apply(transform_title)

Me parece redundante tener 2 variables para decir lo mismo, asi que voy a optar por sacar o el id o el city name

In [ ]:
final_df[[col for col in final_df if col.startswith("seller_address")]].head()

In [ ]:
final_df[[col for col in final_df if col.startswith("seller_address")]].nunique()

Parece ser que: 
- Para la ciudad, el nombre tiene mas informacion
- Para la provincia, es indistinto
- Para el barrio del search location el id es mas representativo
- Para la ciudad del search location el id es mas representativo
- Para la provincia del seach location, es indistinto

In [ ]:
unwanted_seller = ["seller_address.city.id", "seller_address.state.name", "seller_address.search_location.neighborhood.name", 
                   "seller_address.search_location.city.name", "seller_address.search_location.state.name"]
final_df.drop(unwanted_seller, axis=1, inplace = True)

La unica coumna con nombre que nos quedo es para la ciudad del vendedor, pasemosla toda a minuscula y eliminemos whitespaces

In [ ]:
final_df.loc[:, 'seller_address.city.name'] = final_df['seller_address.city.name'].apply(lambda x: x.lower().strip())

Procesemos la informacion que esta adentro de tags

In [ ]:
final_df["tags"] = final_df["tags"].apply(lambda x: " ".join(x)).apply(lambda x: x.replace("-", "_"))
cvect = CountVectorizer()
tags = cvect.fit_transform(final_df["tags"]).toarray()
for i, col in enumerate(cvect.get_feature_names()):
    final_df[col] = tags[:, i]
final_df.drop('tags', axis=1, inplace=True)

Obtengamos los atributos espeificos de la publicacion. Para esto, voy a extraer del diccionario el ID del atributo como su valor. Elijo el value_name y no el value_id por azar, me parecia mas intuitvo para entender y ver la importancia de la feature dado que los puedo asociar

In [ ]:
attrs_df = pd.json_normalize(final_df["attributes"].apply(lambda lista: {d["id"]: d["value_name"] for d in lista}))

In [ ]:
dataset = pd.concat([final_df, attrs_df], axis=1)
dataset.drop("attributes", axis=1, inplace=True)

Un **dato de color** que me hizo **perder unas cuantas horas.....**

In [ ]:
final_df.shape

In [ ]:
attrs_df.shape

In [ ]:
dataset.shape

Por algun motivo que desconzco, el join genero 12 observaciones con todo NA a pesar de que deberia haber hecho el join por indice como corresponde, eliminemoslas

In [ ]:
dataset = dataset[~dataset["sold_quantity"].isna()]

In [ ]:
dataset.shape

In [ ]:
dataset.to_csv("dataset.csv", index=None)
#dataset = pd.read_csv("dataset.csv")

## **Estimador**

Separamos nuestras variables, hagamos una pequeña prueba para ver si tomando logaritmos mejoramos el fit del modelo.

In [ ]:
y = dataset['sold_quantity']
y_log = np.log(dataset['sold_quantity'] + 1)
X = dataset.drop('sold_quantity', axis=1)

Redefinimos nuestras features numericas y categoricas y transformamos el tipo para asegurarnos de no tener errores

In [ ]:
numeric_features = ["price", "discount", "available_quantity", "initial_quantity", 
                    "original_price", "days_old", "days_remaining", "days_from_update"]
X.loc[:, numeric_features] = X[numeric_features].fillna(0)
X[numeric_features] = X[numeric_features].applymap(np.float)
categorical_features = list(X.drop(numeric_features, axis=1).columns)
X.loc[:, categorical_features] = X[categorical_features].fillna("__")
X[categorical_features] = X[categorical_features].applymap(str)

Hagamos la prueba de la transformacion logaritimica usando nuestras variables numericas como predictores.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X[numeric_features], y)

In [ ]:
params = dict(n_estimators=[800, 1000], 
              learning_rate=[0.05, 0.1],
              num_leaves=[20, 30])
estimator = GridSearchCV(
        LGBMRegressor(n_jobs=-1),
        params,
        cv=5,
        n_jobs=-1,
        scoring="r2",
    )
model = estimator.fit(X_train, y_train)

In [ ]:
model.best_params_

In [ ]:
cv = KFold(n_splits=5)
scores = cross_val_score(model, X_test, y_test, cv = cv, scoring='r2')

In [ ]:
scores.mean()

Pareceria funcionar bastante bien, probemos tomando el logaritmo

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X[numeric_features], y_log)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.best_params_

In [ ]:
cv = KFold(n_splits=5)
scores = cross_val_score(model, X_test, y_test, cv = cv, scoring='r2')

In [ ]:
scores.mean()

Hay una mejora tomando el logaritmo. Ahora probemos un modelo con todas las features y veamos como fitea

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_log)

In [ ]:
numeric_transformer = Pipeline(
    [('imputer', SimpleImputer(strategy='constant', fill_value=0))]
)
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='__')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))]
)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
params = dict(n_estimators=[1000, 1500], 
              learning_rate=[0.1],
              num_leaves=[30])
estimator = GridSearchCV(
        LGBMRegressor(n_jobs=-1),
        params,
        cv=5,
        n_jobs=-1,
        scoring="r2",
    )
pipe = Pipeline([('preprocessor', preprocessor), ('estimator', estimator)])
pipe.fit(X_train, y_train)

In [ ]:
pipe.steps[1][1].best_params_

In [ ]:
cv = KFold(n_splits=5)
scores = cross_val_score(pipe, X_test, y_test, cv = cv, scoring='r2')

In [ ]:
scores.mean()

No pareceria haber una mejora significativa por agregar la inmensa cantidad de features categoricos. De todas maneras, por todo el trabajo hecho, voy a optar por seguir el analisis de este modelo en general que, supongo, puede ser mas extensible a un predictor general de items.

In [ ]:
ohe_feature_names = pipe.steps[0][1].transformers_[1][1].steps[1][1].get_feature_names(categorical_features)
feature_importances = pipe.steps[1][1].best_estimator_.feature_importances_

cols = list(numeric_features) + list(ohe_feature_names)
f_i = list(zip(cols, feature_importances))
feat_imp = pd.DataFrame(f_i, columns=["feature_names", "importance"])
feat_imp = feat_imp.sort_values('importance', ascending=False).reset_index(drop=True)

In [ ]:
feat_imp[:15]

Como esperabamos, las features numericas son las mas relevantes al momento de explicar la cantidad de ventas

In [ ]:
preds_df = X_test.copy()
preds = pipe.predict(X_test)

Transformamos nuestros datos, recodemos que estan en logartimos, para analizar los resultados

In [ ]:
preds_df['preds'] = np.round(np.exp(preds) - 1)
preds_df['real'] = np.exp(y_test) - 1
preds_df["error"] = np.abs(preds_df['real'] - preds_df['preds'])

Fijemosnos como fitea nuestro modelo

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x="preds", y="real", data=preds_df)
x = [x for x in range(ceil(preds_df['preds'].max()))]
plt.plot(x, x);

Pareceria fittear bastante bien, veamos en valores de ventas mas chicos

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x="preds", y="real", data=preds_df)
x = [x for x in range(ceil(preds_df['preds'].max()))]
plt.plot(x, x)
plt.xlim(0, 1000)
plt.ylim(0, 2000);

Podemos ver que hay casos donde el error es significativo, hay un dato cuyo valor real es ~1100 y estamos prediciendo casi 10. Mas adelante nos vamos a concentrar en estos casos. Vayamos un paso mas, y veamos como fittea en valores aun mas chicos

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x="preds", y="real", data=preds_df)
x = [x for x in range(ceil(preds_df['preds'].max()))]
plt.plot(x, x)
plt.xlim(0, 200);
plt.ylim(0, 500);

Podemos ver que hay errores interesantes a analizar

Veamos si el error se asocia a algun tipo en proudcto en particular

In [ ]:
preds_df.groupby("domain_id").mean().reset_index().plot.bar(x="domain_id", y='error', title='Error promedio por domain_id', figsize=(12,8))
plt.xticks(rotation=70);

Podemos ver que performa peor para auriculares y parlantes. Fijemosnos que paso con las observaciones que tenian errores muy grandes

In [ ]:
target_cols = ["title", "domain_id", "seller_id", "category_id"]
target_cols += feat_imp['feature_names'][:6].values.tolist() 
target_cols += ["real", "error"]

In [ ]:
errors_df = preds_df.query("error > 200")[target_cols].sort_values("error", ascending=False)

In [ ]:
errors_df[:10]

Es dificil encontrar a ojo una relacion entre el error y nuestras features aunque se puede ver que el valor real correlaciona bastante bien con el error. Es de esperar ya que la mayoria de nuestras observaciones tienen pocas ventas

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(errors_df.corr(), 
        xticklabels=errors_df.corr().columns,
        yticklabels=errors_df.corr().columns,
        center=0,
        cmap=sns.diverging_palette(220, 20, as_cmap=True));

Verificamos que correlaciona positivamente con el total de ventas

Podriamos seguir haciendo un analisis de inferencia causal con regresiones, significancia estadistica, intervalos de confianza, etc, para determinar las features que explican el error pero voy a optar por dejarlo aca ya que nuestro mayor problema es la asimetria en la distribucion de ventas.

## **Conclusion**

Conseguimos un modelo bastante acertado para publicaciones de computadores, parlantes, celulares, monitores y notebooks. Un modelo con solo las variables numericas, para este contexto, parece ser la mejor opcion dada su liviandad de entrenamiento y carencia de preprocesamiento. De todas maneras, para predecir publicaciones en toda la categoria creo que las features categories pueden ser muy utiles dado que vemos (evaluando feature importance) que aportan informacion. Sortenado las dificultades de infraestructura, confio que el modelo puede funcionar para publiciaciones en general.